# Imports

In [5]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize spark context

In [6]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "2")
         .config("spark.executor.memory", "10g")
         .config("spark.executor.cores", "4")
         .config("spark.driver.memory", "10g")
         .config("spark.cores.max", "12")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

'http://69abb479a5e1:4041'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [12]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets_stanza/')
seleccion.filtrar_sentencias(preprocess=solo_portanto)

estructurales = SentenciasEstructurales(seleccion)

# Formar dataset de sentencias estructurales

In [13]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, spacy=False, actualizar_sdf=True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()

2021-09-23 22:29:28 INFO: Loading these models for language: es (Spanish): / 27]
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-09-23 22:29:28 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-09-23 22:29:28 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-09-23 22:29:28 INFO: Use device: gpu
2021-09-23 22:29:28 INFO: Loading: tokenize
2021-09-23 22:29:28 INFO: Use device: gpu
2021-09-23 22:29:28 INFO: Loading: tokenize
2021-09-23 22:29:2

Py4JJavaError: An error occurred while calling o340.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apac

# Cargar procesamiento de sentencias estructurales.

In [4]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
estructurales.seleccion.sdf.limit(10).toPandas()

,archivo,txt,cleanText,num,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,tema,...,se_ordena,plan,plazo,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,extension_sentencia,extension_por_lo_tanto
0,96AF8.html,\n* 080147960007CO * Exp: 08-014796-0007-CO Re...,co exp co res sal constitucional cort suprem j...,132,2009-02-13 13:27:00,Abdelnour Granados,Recurso de Amparo,Con lugar,False,TRABAJO,...,1,0,0,None,None,None,None,None,19878,1641
1,96006.html,\n* 090051320007CO * Exp: 09-005132-0007-CO Re...,co exp co res sal constitucional cort suprem j...,133,2009-04-30 18:33:00,Armijo Sancho,Recurso de Amparo,Con lugar,False,TRABAJO,...,1,0,0,[Ricardo Vindas Valerio],None,[Director de Recursos Humanos],None,None,12682,1597
2,9A831.html,\n*090057240007CO* Exp: 09-005724-0007-CO Res....,co exp co res sal constitucional cort suprem j...,216696,NaT,Araya García,Recurso de Amparo,Con lugar parcial,False,PETICION,...,1,0,1,"[Miguel A. Carabaguíaz Murillo, Carlos Fernand...",None,"[Presidente Ejecutivo, Instituto Costarricense...",None,None,25425,1917
3,9ED23.html,\n*090035170007CO* Exp: 09-003517-0007-CO Res....,co exp co res sal constitucional cort suprem j...,216698,2009-05-08 11:29:00,None,Recurso de Amparo,Con lugar,False,TRABAJO,...,1,0,0,[Fernando Bogantes Cruz],None,[Director de Recursos Humanos del Ministerio d...,None,None,7186,1672
4,94C54.html,\n* 080177720007CO * Exp: 08-017772-0007-CO Re...,co exp co res sal constitucional cort suprem j...,216704,2009-03-17 11:36:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,TRABAJO,...,1,0,0,"[Fernando Bogantes Cruz, FRAN ARROYO PEÑA]",[Zona de Menor Desarrollo],[Director de Recursos Humanos del Ministerio d...,None,None,27198,1507
5,95058.html,\n* 090000300007CO * Exp: 09-000030-0007-CO Re...,co exp co res sal constitucional cort suprem j...,216716,2009-03-18 15:03:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,TRABAJO,...,1,0,0,"[Fernando Bogantes Cruz, ALVARADO DÍAZ]",[Zona de Menor Desarrollo],[Director de Recursos Humanos del Ministerio d...,None,None,27096,1518
6,93B9B.html,\n* 080169410007CO * Exp: 08-016941-0007-CO Re...,co exp co res sal constitucional cort suprem j...,216717,2009-02-17 11:34:00,Calzada Miranda,Recurso de Amparo,Con lugar,False,TRABAJO,...,1,0,0,[Fernando Bogantes Cruz],[Zona de Menor Desarrollo],[Director de Recursos Humanos del Ministerio d...,None,None,27174,1516
7,9D741.html,\n*080170600007CO* Exp: 08-017060-0007-CO Res....,co exp co res sal constitucional cort suprem j...,216719,2009-01-30 12:41:00,Cruz Castro,Recurso de Amparo,Con lugar,False,TRABAJO,...,0,0,1,None,None,None,None,None,5794,1745
8,941A8.html,\n* 080169340007CO * Exp: 08-016934-0007-CO Re...,co exp co res sal constitucional cort suprem j...,216720,2009-02-17 10:02:00,Calzada Miranda,Recurso de Amparo,Con lugar,False,TRABAJO,...,1,0,0,"[Fernando Bogantes Cruz, GEORGINA VARGAS PEREIRA]",[Zona de Menor Desarrollo],[Director de Recursos Humanos del Ministerio d...,None,None,27167,1520
9,9979B.html,\n*090053520007CO* EXPEDIENTE N° 09-005352-000...,co expedient co proces recurs ampar resolu sal...,216728,2009-07-21 19:24:00,Calzada Miranda,Recurso de Amparo,Con lugar parcial,False,TRABAJO,...,1,0,0,"[Alberto Orozco Canossa, ERNESTO CHAVERRI VARGAS]",[Zona de Menor Desarrollo],[Director de Recursos Humanos del Ministerio d...,None,None,26114,1609


In [ ]:
estructurales.seleccion.sdf.toPandas()[estructurales.seleccion.sdf.toPandas()['plazo'] > 0]['txt'].iloc[[1]]

In [ ]:
print(estructurales.seleccion.sdf.filter("expediente == '170020150007CO'").select("txt").toPandas().loc[0,'txt'])

In [ ]:
from spacy.matcher import Matcher

txt = """ Se declara con lugar el recurso. Se ordena a Yaxinia Díaz Mendoza, en calidad de Directora de Recursos Humanos, a Rosa Adolio Cascante, en calidad de Viceministra Administrativa y a Walter Muñoz Caravaca en su calidad de Director de Infraestructura y Equipamiento, todos del Ministerio de Educación Pública o a quienes en su lugar ocupen los cargos, cada uno dentro del ámbito de sus competencias, que en el plazo máximo de OCHO DÍAS contados a partir de la notificación de esta sentencia realicen el trámite correspondiente para nombrar un Profesor de Enseñanza General Básica 1 en la Escuela La Katira. Deberán además en el plazo máximo de TRES MESES contados a partir de la notificación de esta sentencia iniciar con los procedimientos necesarios –procedimientos que incluyen tanto la aprobación de presupuesto como el proceso de contratación- para la construcción de la infraestructura requerida por la Escuela La Katira. En ambas órdenes se debe garantizar a los estudiantes perjudicados su continuidad en el proceso educativo. Se advierte a dichos funcionarios que de no acatar la orden dicha, incurrirán en el delito de desobediencia y, que de conformidad con el artículo 71 de la Ley de esta jurisdicción, se le impondrá prisión de tres meses a dos años, o de veinte a sesenta días multa, a quien recibiere una orden que deba cumplir o hacer cumplir, dictada en un recurso de amparo y no la cumpliere o no la hiciere cumplir, siempre que el delito no esté más gravemente penado. Se condena al Estado al pago de las costas, daños y perjuicios causados con los hechos que sirven de base a esta declaratoria, los que serán liquidados en ejecución de sentencia de lo contencioso administrativo. El Magistrado Salazar Alvarado pone nota. La Magistrada Esquivel Rodríguez consigna nota. Notifíquese la presente resolución a Yaxinia Díaz Mendoza, en calidad de Directora de Recursos Humanos, a Rosa Adolio Cascante, en calidad de Viceministra Administrativa y a Walter Muñoz Caravaca en su calidad de Director de Infraestructura y Equipamiento, todos del Ministerio de Educación Pública o a quienes en su lugar ocupen los cargos, EN FORMA PERSONAL. Deberán las autoridades recurridas informar a esta Sala sobre el avance en la construcción de las aulas en la Escuela La Katira. Comuníquese al Ministro de Educación Pública el contenido de esta sentencia.
"""
patron = [{"LOWER": "plazo"},
          {"TEXT": {"REGEX": "^(?!\.|[Hh][OoÓó][Rr][Aa]([Ss])?|[dD][iíIÍ][ÁAaa]([Ss])?|[Mm][EeÉé][Ss]([EeÉé][Ss])?|[ÁáAa][Ññ][OoÓó]([Ss])?)"}, "OP": "+"},
          {"TEXT": {"REGEX": "\.|[Hh][OoÓó][Rr][Aa]([Ss])?|[dD][iíIÍ][ÁAaa]([Ss])?|[Mm][EeÉé][Ss]([EeÉé][Ss])?|[ÁáAa][Ññ][OoÓó]([Ss])?"}},
          ]

patterns = [patron]

nlp = spark_get_spacy('es_core_news_lg')
doc = nlp(txt)
matcher = Matcher(nlp.vocab)
matcher.add("Patron 1 :", patterns, greedy="FIRST")

matches = matcher(doc)
print(matches)
plazos = []
for match_id, start, end in matches:
    includeText = False
    plazo = ""
    for token in doc[start:end]:
        if includeText:
            if token.pos_ == "PUNCT":
                break
            plazo += " " + token.text
        else:
            if token.pos_ == "NUM":
                plazo += token.text
                includeText = True
    if plazo != "":
        plazos.append(plazo)
                
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    print("SPAN")
    span =  doc[start:end]
    print(span.text)

print("PLAZOS")
print(plazos)

In [ ]:
estructurales.sdf.toPandas()[estructurales.sdf.toPandas()['expediente'] == '040099160007CO']

In [ ]:
from pyspark.sql.types import *
columnas = {
    'plazosDefinidos' : ArrayType(StringType())
}
resultado = estructurales.plazosDefinidos(columnas)
resultado.toPandas()

In [ ]:
resultado.toPandas()[50:100]

In [ ]:
plazo = ""
plazo = plazo + " perro "
plazo = plazo + " gato "
print(plazo)

In [ ]:

print(pd.Timedelta('2 month'))

In [ ]:
t = "9:05"
print(t.isalpha())